In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math

In [3]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
file_directory = '/content/drive/MyDrive/momentum-strategies/technical_indicators'

In [4]:
qqq_df = pd.read_csv(f'{file_directory}/QQQ_daily161118_211117.csv')

In [5]:
tqqq_df = pd.read_csv(f'{file_directory}/TQQQ_daily161118_211117.csv')

In [51]:
class Portfolio:
  def __init__(self, benchmark_df, benchmark_name):
    self.df = benchmark_df.copy()
    self.df.drop(['Adj Close', 'Volume'], axis=1, inplace=True)
    self.benchmark_closes = benchmark_df['Close'].to_numpy()
    self.benchmark_opens = benchmark_df['Open'].to_numpy()
    self.benchmark_highs = benchmark_df['High'].to_numpy()
    self.benchmark_lows = benchmark_df['Low'].to_numpy()
    self.benchmark_name = benchmark_name
  
  def init_config(self, initial_cash=10000, dc_lengths=[20,55], atr_length=20):
    self.df_columns = [
      'rsi',
      'buy_price', 'sell_price', 'shares', 'position_size', 'cash', 
    ]
    for column in self.df_columns:
      setattr(self, column, np.zeros(len(self.benchmark_closes)))
    self.initial_cash = initial_cash

  def get_rsi(self, window_length):
    # Calculate the RSI based on MMA
    delta = self.df['Close'].diff()
    # first row is NaN since it did not have a previous row to calculate the differences
    up, down = delta.clip(lower=0), delta.clip(upper=0).abs()
    # Make the positive gains (up) and negative gains (down) Series
    alpha = 1 / window_length
    roll_up = up.ewm(alpha=alpha).mean()
    roll_down = down.ewm(alpha=alpha).mean()
    rs = roll_up / roll_down
    rsi_mma = 100.0 - (100.0 / (1.0 + rs))
    self.rsi = rsi_mma.to_numpy()

  def save_to_df(self, to_csv=False):
    for column in self.df_columns:
      self.df[column] = getattr(self, column)
    self.df['portfolio_closes'] = self.cash + self.shares * self.benchmark_closes
    if to_csv:
      self.df.to_csv(f'{file_directory}/{current_date}_{self.benchmark_name}_turtle.csv')

In [50]:
ptfl = Portfolio(qqq_df, 'qqq')
ptfl.init_config()
ptfl.get_rsi(14)
ptfl.rsi

array([         nan, 100.        , 100.        , ...,  65.61370639,
        68.30406124,  68.50238677])